In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!cp /content/drive/MyDrive/neuromarketing/dataset_sd.pkl ./dataset_sd.pkl

# 0. Package Import

In [23]:
import pickle
import numpy as np
import random
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, DepthwiseConv2D, AveragePooling2D, Dropout, SeparableConv2D, LeakyReLU, Reshape, Flatten, Dense, Softmax, Input, Bidirectional, LSTM, Multiply, Concatenate, GlobalAveragePooling1D, GlobalAveragePooling2D, ELU
from tensorflow.keras.constraints import max_norm
import os
from time import time

In [5]:
# Fixing seed for reproduction of results
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
tf.random.set_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

# GPU availability
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


# 1. Model Implementation

In [6]:
class CNNBiLSTM(Model):
  def __init__(self):
    super(CNNBiLSTM, self).__init__()
    self.reshape = Reshape((14, 512, 1), input_shape=(14, 512))
    self.reshape2 = Reshape((512, 14), input_shape=(14, 512))
    self.conv1 = Conv2D(8, (1, 64), padding='same', input_shape=(14, 512, 1), use_bias=False)
    self.bn1 = BatchNormalization()
    self.bn2 = BatchNormalization()
    self.bn3 = BatchNormalization()
    self.bn4 = BatchNormalization()
    self.conv2 = DepthwiseConv2D((14, 1), use_bias=False, depth_multiplier=2, depthwise_constraint=max_norm(1.))
    self.activation = ELU() # LeakyReLU()
    self.pool1 = AveragePooling2D(1, 4)
    self.dropout = Dropout(0.25)
    self.conv3 = SeparableConv2D(16, (1, 16), use_bias=False, padding='same')
    self.pool2 = AveragePooling2D(1, 8)
    self.flatten = Flatten()
    self.fc = Dense(1, activation='sigmoid')
    self.bilstm = Bidirectional(LSTM(128, dropout=0.25, return_sequences=False), input_shape=(512, 14))
    self.glu_fc = Dense(256)
    self.glu_sig = Dense(256, activation='sigmoid')
    self.mul = Multiply()
    self.concat = Concatenate(axis=-1)
    self.gap1d = GlobalAveragePooling1D()

  def call(self, x):
    c = self.reshape(x)
    c = self.bn1(self.conv1(c))
    c = self.dropout(self.pool1(self.activation(self.bn2(self.conv2(c)))))
    c = self.flatten(self.dropout(self.pool2(self.activation(self.bn3(self.conv3(c))))))
    l = self.reshape2(x)
    l = self.bilstm(l)
    lc = self.concat([l, c])
    lc = self.glu_fc(lc) * self.glu_sig(lc)
    lc = self.fc(self.dropout(self.activation(self.bn4(lc))))
    return lc

  def summary(self):
    inputs = Input((14, 512))
    Model(inputs, self.call(inputs)).summary()

In [8]:
model = CNNBiLSTM()
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 14, 512)]    0           []                               
                                                                                                  
 reshape (Reshape)              (None, 14, 512, 1)   0           ['input_1[0][0]']                
                                                                                                  
 conv2d (Conv2D)                (None, 14, 512, 8)   512         ['reshape[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 14, 512, 8)  32          ['conv2d[0][0]']                 
 alization)                                                                                   

# 2. Training, Grid search, and Evaluation

In [11]:
with open('./dataset_sd.pkl', 'rb') as f:
  data_cv = pickle.load(f)

In [10]:
class run(object):
  def __init__(self, name, model, batch_size, lr, epoch, patience, verbose):
    self.batch_size = batch_size
    self.name = name
    self.lr = lr
    self.epoch = epoch
    self.patience = patience
    self.verbose = verbose
    self.model = model
    self.time_taken = None
    self.model.compile(optimizer=tf.keras.optimizers.experimental.Adam(learning_rate=self.lr), loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy', tf.keras.metrics.AUC()])
    self.callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=self.patience, verbose=0, mode='auto', baseline=None, restore_best_weights=True)

  def fit(self, X_train, y_train, X_val, y_val):
    self.start_time = time()
    self.model.fit(X_train, y_train, epochs=self.epoch, batch_size=self.batch_size, verbose=self.verbose, validation_data=(X_val, y_val), callbacks=[self.callbacks], shuffle=True)
    self.time_taken = time() - self.start_time

  def eval(self, X, y):
    test_loss, test_acc, test_auc = self.model.evaluate(X, y, verbose=self.verbose)
    print('Setting: model={}, batch_size={}, lr={}, epoch={}, patience={} || loss={}, acc={}, auc={} || training_time={} sec'.format(self.name, self.batch_size, self.lr, self.epoch, self.patience, test_loss, test_acc, test_auc, self.time_taken))

In [9]:
class loocv_run(object):
  def __init__(self, name, model, batch_size, lr, epoch, patience, verbose, data):
    self.batch_size = batch_size
    self.name = name
    self.lr = lr
    self.epoch = epoch
    self.patience = patience
    self.verbose = verbose
    self.model = model
    self.time_taken = None
    # self.model.compile(optimizer=tf.keras.optimizers.experimental.Adam(learning_rate=self.lr), loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy', tf.keras.metrics.AUC()])
    self.callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=self.patience, verbose=0, mode='auto', baseline=None, restore_best_weights=True)
    self.data = data
    self.acc = 0
    self.auc = 0

  def _fit(self, X_train, y_train, X_test, y_test, subject):
    model = self.model
    model.compile(optimizer=tf.keras.optimizers.experimental.Adam(learning_rate=self.lr), loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy', tf.keras.metrics.AUC()])
    self.start_time = time()
    model.fit(X_train, y_train, epochs=self.epoch, batch_size=self.batch_size, verbose=self.verbose, validation_split=0.1, callbacks=[self.callbacks], shuffle=True)
    self.time_taken = time() - self.start_time
    test_loss, test_acc, test_auc = model.evaluate(X_test, y_test, verbose=self.verbose)
    self.acc += test_acc
    self.auc += test_auc
    print('Setting: subject={}, batch_size={}, lr={}, epoch={}, patience={} || loss={}, acc={}, auc={} || training_time={} sec'.format(subject, self.batch_size, self.lr, self.epoch, self.patience, test_loss, test_acc, test_auc, self.time_taken))
  
  def loocv(self):
    for k, v in self.data.items():
      X_test, y_test = self.data[k]
      X_train = None
      y_train = None
      cnt = 0
      for key in self.data.keys():
        if k != key:
          cnt+=1
          if cnt==1:
            X_train = self.data[key][0]
            y_train = self.data[key][1]
          X_train = np.append(X_train, self.data[key][0], axis=0)
          y_train = np.append(y_train, self.data[key][1], axis=0)
      self._fit(X_train, y_train, X_test, y_test, k)
      tf.keras.backend.clear_session()
    print('Avg_ACC={}, Avg_AUC={}'.format(self.acc/25, self.auc/25))

In [19]:
r = loocv_run(name='CNNBiLSTM', model=CNNBiLSTM(), batch_size=16, lr=0.001, epoch=200, patience=50, verbose=0, data=data_cv)
r.loocv()

Setting: subject=Abhishek, batch_size=16, lr=0.001, epoch=200, patience=50 || loss=0.7219246625900269, acc=0.5, auc=0.4285714626312256 || training_time=115.27562761306763 sec
Setting: subject=Ankur_sir, batch_size=16, lr=0.001, epoch=200, patience=50 || loss=0.6774824857711792, acc=0.6097561120986938, auc=0.648809552192688 || training_time=113.63859796524048 sec
Setting: subject=Gautam, batch_size=16, lr=0.001, epoch=200, patience=50 || loss=0.5121483206748962, acc=0.773809552192688, auc=0.7819445133209229 || training_time=108.89583230018616 sec
Setting: subject=Gautam_123, batch_size=16, lr=0.001, epoch=200, patience=50 || loss=0.3557807505130768, acc=0.9047619104385376, auc=0.9814815521240234 || training_time=116.42300510406494 sec
Setting: subject=Girvar_yadav, batch_size=16, lr=0.001, epoch=200, patience=50 || loss=0.5465646982192993, acc=0.6904761791229248, auc=0.7941176295280457 || training_time=113.99435496330261 sec
Setting: subject=Kishore_babu, batch_size=16, lr=0.001, epoch=

# 3. Baseline Classifier

In [7]:
class EEGNet(Model):
  def __init__(self):
    super(EEGNet, self).__init__()
    self.reshape = Reshape((14, 512, 1), input_shape=(14, 512))
    self.conv1 = Conv2D(8, (1, 64), padding='same', input_shape=(14, 512, 1), use_bias=False)
    self.bn1 = BatchNormalization()
    self.bn2 = BatchNormalization()
    self.bn3 = BatchNormalization()
    self.conv2 = DepthwiseConv2D((14, 1), use_bias=False, depth_multiplier=2, depthwise_constraint=max_norm(1.))
    self.activation = ELU()
    self.pool1 = AveragePooling2D(1, 4)
    self.dropout = Dropout(0.25)
    self.conv3 = SeparableConv2D(16, (1, 16), use_bias=False, padding='same')
    self.pool2 = AveragePooling2D(1, 8)
    self.flatten = Flatten()
    self.fc = Dense(1, activation='sigmoid')
    # self.softmax = Softmax()

  def call(self, x):
    x = self.reshape(x)
    x = self.bn1(self.conv1(x))
    x = self.dropout(self.pool1(self.activation(self.bn2(self.conv2(x)))))
    x = self.dropout(self.pool2(self.activation(self.bn3(self.conv3(x)))))
    x = self.fc(self.flatten(x))
    return x

  def summary(self):
    inputs = Input((14, 512))
    Model(inputs, self.call(inputs)).summary()

In [ ]:
EEGNet().summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 14, 512)]    0           []                               
                                                                                                  
 reshape_3 (Reshape)            (None, 14, 512, 1)   0           ['input_1[0][0]']                
                                                                                                  
 conv2d_3 (Conv2D)              (None, 14, 512, 8)   512         ['reshape_3[0][0]']              
                                                                                                  
 batch_normalization_9 (BatchNo  (None, 14, 512, 8)  32          ['conv2d_3[0][0]']               
 rmalization)                                                                                 

In [12]:
r = loocv_run(name='EEGNet', model=EEGNet(), batch_size=16, lr=0.001, epoch=200, patience=50, verbose=0, data=data_cv)
r.loocv()

Setting: subject=Abhishek, batch_size=16, lr=0.001, epoch=200, patience=50 || loss=0.6855916380882263, acc=0.5714285969734192, auc=0.5748299360275269 || training_time=28.89529514312744 sec
Setting: subject=Ankur_sir, batch_size=16, lr=0.001, epoch=200, patience=50 || loss=0.6316110491752625, acc=0.707317054271698, auc=0.7690476179122925 || training_time=19.704755783081055 sec
Setting: subject=Gautam, batch_size=16, lr=0.001, epoch=200, patience=50 || loss=0.626918613910675, acc=0.6785714030265808, auc=0.6975694894790649 || training_time=18.82360076904297 sec
Setting: subject=Gautam_123, batch_size=16, lr=0.001, epoch=200, patience=50 || loss=0.6182053685188293, acc=0.5952380895614624, auc=0.7246913909912109 || training_time=19.84825825691223 sec
Setting: subject=Girvar_yadav, batch_size=16, lr=0.001, epoch=200, patience=50 || loss=0.5895609259605408, acc=0.738095223903656, auc=0.7964706420898438 || training_time=19.392816066741943 sec
Setting: subject=Kishore_babu, batch_size=16, lr=0.

In [13]:
class BiLSTM(Model):
  def __init__(self):
    super(BiLSTM, self).__init__()
    self.reshape = Reshape((512, 14), input_shape=(14, 512))
    self.bilstm = Bidirectional(LSTM(128), input_shape=(512, 14))
    self.flatten = Flatten()
    # self.gap1d = # GlobalAveragePooling1D()
    self.fc = Dense(1, activation='sigmoid')
    # self.softmax = Softmax()

  def call(self, x):
    x = self.reshape(x)
    x = self.bilstm(x)
    x = self.fc(self.flatten(x))
    return x

  def summary(self):
    inputs = Input((14, 512))
    Model(inputs, self.call(inputs)).summary()

In [14]:
r = loocv_run(name='BiLSTM', model=BiLSTM(), batch_size=16, lr=0.001, epoch=200, patience=50, verbose=0, data=data_cv)
r.loocv()

Setting: subject=Abhishek, batch_size=16, lr=0.001, epoch=200, patience=50 || loss=0.677507758140564, acc=0.5714285969734192, auc=0.6054421663284302 || training_time=112.2715539932251 sec
Setting: subject=Ankur_sir, batch_size=16, lr=0.001, epoch=200, patience=50 || loss=0.5106298327445984, acc=0.8048780560493469, auc=0.8428571224212646 || training_time=99.53924322128296 sec
Setting: subject=Gautam, batch_size=16, lr=0.001, epoch=200, patience=50 || loss=0.4083179831504822, acc=0.8333333134651184, auc=0.8694444894790649 || training_time=95.94277167320251 sec
Setting: subject=Gautam_123, batch_size=16, lr=0.001, epoch=200, patience=50 || loss=0.3321716785430908, acc=0.8809523582458496, auc=0.9518517851829529 || training_time=101.48081588745117 sec
Setting: subject=Girvar_yadav, batch_size=16, lr=0.001, epoch=200, patience=50 || loss=0.6405085921287537, acc=0.6666666865348816, auc=0.7341176271438599 || training_time=146.0391447544098 sec
Setting: subject=Kishore_babu, batch_size=16, lr=0

In [24]:
class lstm(Model):
  def __init__(self):
    super(lstm, self).__init__()
    self.reshape = Reshape((512, 14), input_shape=(14, 512))
    self.lstm = LSTM(128)
    self.flatten = Flatten()
    # self.gap1d = # GlobalAveragePooling1D()
    self.fc = Dense(1, activation='sigmoid')
    # self.softmax = Softmax()

  def call(self, x):
    x = self.reshape(x)
    x = self.lstm(x)
    x = self.fc(self.flatten(x))
    return x

  def summary(self):
    inputs = Input((14, 512))
    Model(inputs, self.call(inputs)).summary()

In [25]:
r = loocv_run(name='LSTM', model=lstm(), batch_size=16, lr=0.001, epoch=200, patience=50, verbose=0, data=data_cv)
r.loocv()

Setting: subject=Abhishek, batch_size=16, lr=0.001, epoch=200, patience=50 || loss=0.611602783203125, acc=0.7142857313156128, auc=0.761904776096344 || training_time=63.45330619812012 sec
Setting: subject=Ankur_sir, batch_size=16, lr=0.001, epoch=200, patience=50 || loss=0.6292135119438171, acc=0.6585366129875183, auc=0.7321428060531616 || training_time=60.848843812942505 sec
Setting: subject=Gautam, batch_size=16, lr=0.001, epoch=200, patience=50 || loss=0.3814341723918915, acc=0.8571428656578064, auc=0.9277777671813965 || training_time=53.92619466781616 sec
Setting: subject=Gautam_123, batch_size=16, lr=0.001, epoch=200, patience=50 || loss=0.363726943731308, acc=0.8333333134651184, auc=0.9358024001121521 || training_time=57.29503631591797 sec
Setting: subject=Girvar_yadav, batch_size=16, lr=0.001, epoch=200, patience=50 || loss=0.5854266881942749, acc=0.6428571343421936, auc=0.774117648601532 || training_time=57.5116662979126 sec
Setting: subject=Kishore_babu, batch_size=16, lr=0.001

In [28]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier

def ml_run(name, model, X_train, y_train, X_test, y_test):
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  acc = accuracy_score(y_test, y_pred)
  AUC = roc_auc_score(y_test, y_pred)
  print("**Subject: {} || Acc: {}, AUC: {}".format(name, acc, AUC))
  return acc, AUC

def ml_cv_run(name, model, data):
  acc = 0
  auc = 0
  for k, v in data.items():
        X_test, y_test = data[k]
        X_train = None
        y_train = None
        cnt = 0
        for key in data.keys():
          if k != key:
            cnt+=1
            if cnt==1:
              X_train = data[key][0]
              y_train = data[key][1]
            X_train = np.append(X_train, data[key][0], axis=0)
            y_train = np.append(y_train, data[key][1], axis=0)
        m = model
        Acc, AUC = ml_run(k, m, X_train.reshape(X_train.shape[0], -1), y_train.reshape(y_train.shape[0], -1), X_test.reshape(X_test.shape[0], -1), y_test.reshape(y_test.shape[0], -1))
  print('Avg_ACC={}, Avg_AUC={}'.format(acc/25, auc/25))

In [31]:
# SVM
ml_cv_run('SVM', SVC(), data_cv)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


**Subject: Abhishek || Acc: 0.5476190476190477, AUC: 0.5476190476190477


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


**Subject: Ankur_sir || Acc: 0.5609756097560976, AUC: 0.55


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


...

**Subject: Vijay || Acc: 0.5714285714285714, AUC: 0.5


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


**Subject: Vipin_1 || Acc: 0.4523809523809524, AUC: 0.45238095238095244


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


**Subject: Viraj_1 || Acc: 0.8095238095238095, AUC: 0.6363636363636364
Avg_ACC=0.0, Avg_AUC=0.0


In [32]:
#RF
ml_cv_run('RF', RandomForestClassifier(), data_cv)

<ipython-input-28-8fe9f1d6d9d5>:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)


**Subject: Abhishek || Acc: 0.4523809523809524, AUC: 0.45238095238095233


<ipython-input-28-8fe9f1d6d9d5>:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)


**Subject: Ankur_sir || Acc: 0.5853658536585366, AUC: 0.575


<ipython-input-28-8fe9f1d6d9d5>:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)


...

**Subject: Vijay || Acc: 0.47619047619047616, AUC: 0.4236111111111111


<ipython-input-28-8fe9f1d6d9d5>:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)


**Subject: Vipin_1 || Acc: 0.47619047619047616, AUC: 0.47619047619047616


<ipython-input-28-8fe9f1d6d9d5>:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)


**Subject: Viraj_1 || Acc: 0.6190476190476191, AUC: 0.41935483870967744
Avg_ACC=0.0, Avg_AUC=0.0


In [33]:
#LDA
ml_cv_run('LDA', LinearDiscriminantAnalysis(), data_cv)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


**Subject: Abhishek || Acc: 0.5476190476190477, AUC: 0.5476190476190477


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


**Subject: Ankur_sir || Acc: 0.5365853658536586, AUC: 0.5380952380952381


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


...

**Subject: Vijay || Acc: 0.5238095238095238, AUC: 0.5138888888888888


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


**Subject: Vipin_1 || Acc: 0.35714285714285715, AUC: 0.35714285714285715


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


**Subject: Viraj_1 || Acc: 0.5, AUC: 0.48533724340175954
Avg_ACC=0.0, Avg_AUC=0.0


In [34]:
#KNN
ml_cv_run('KNN',  KNeighborsClassifier(), data_cv)

/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


**Subject: Abhishek || Acc: 0.5238095238095238, AUC: 0.5238095238095238


/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


**Subject: Ankur_sir || Acc: 0.4878048780487805, AUC: 0.4988095238095238


/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


...

/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


**Subject: Vijay || Acc: 0.40476190476190477, AUC: 0.4513888888888889


/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


**Subject: Vipin_1 || Acc: 0.5476190476190477, AUC: 0.5476190476190477
**Subject: Viraj_1 || Acc: 0.35714285714285715, AUC: 0.564516129032258
Avg_ACC=0.0, Avg_AUC=0.0


/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
